# Module 5: Memory, GIL, & Internal Performance

### **The Scenario:**

You’ve built a data processing pipeline that tracks 1 million delivery trucks in real-time. It worked great on your laptop with 10 trucks. But in production, your server is running out of RAM, and your CPU usage is stuck at 5% despite having 16 cores.


### **The Goal:**
By the end of this module, you will debug these specific crashes by looking inside the Python interpreter.

## Lesson 1: The hidden cost of "Everything is an Object"

### The Problem

Your server logs show a `MemoryError`. You calculated that 1 million GPS points (just two integers, x and y) should take about **16MB**. But in reality, your Python script is eating **180MB**. Why?

### The "Aha!" Moment

Every custom object you create (like a `Truck` class) carries a hidden dictionary (`__dict__`) to store its attributes. This allows for dynamic attribute addition, but it costs significant memory.

### The Solution: `__slots__`

By defining `__slots__`, you tell Python: *"Don't give me a dictionary. Just reserve space for these specific attributes."*

In [20]:
!pip install pympler
import sys
from pympler import asizeof

class BloatedTruck:
    def __init__(self, lat, lng):
        self.lat = lat
        self.lng = lng

class OptimizedTruck:
    __slots__ = ['lat', 'lng']
    def __init__(self, lat, lng):
        self.lat = lat
        self.lng = lng

bloated = BloatedTruck(40.7128, -74.0060)
lean = OptimizedTruck(40.7128, -74.0060)

print(f"Bloated Object Real Size: {asizeof.asizeof(bloated)} bytes")
print(f"Optimized Object Real Size: {asizeof.asizeof(lean)} bytes")
print(f"RAM Savings: {((asizeof.asizeof(bloated) - asizeof.asizeof(lean)) / asizeof.asizeof(bloated)) * 100:.2f}%")


[notice] A new release of pip is available: 25.0.1 -> 26.0
[notice] To update, run: pip install --upgrade pip
Bloated Object Real Size: 312 bytes
Optimized Object Real Size: 96 bytes
RAM Savings: 69.23%


## Lesson 2: Sequence Memory (List vs. Tuple)

### The Problem
You noticed that even when you aren't adding new items, your lists seem to occupy more memory than expected.

### The "Aha!" Moment
Python lists use **Over-allocation**. To make `append()` fast ($O(1)$), Python grabs more memory than it needs so it doesn't have to resize every single time you add an item. Tuples, being immutable, are allocated exactly at the size needed.

In [1]:
import sys

lst = []
print("Watching List growth (Over-allocation in action):")
for i in range(10):
    lst.append(i)
    print(f"Items: {len(lst)} | Size in RAM: {sys.getsizeof(lst)} bytes")

tup = tuple(range(10))
print(f"\nStatic Tuple Size for 10 items: {sys.getsizeof(tup)} bytes")

Watching List growth (Over-allocation in action):
Items: 1 | Size in RAM: 88 bytes
Items: 2 | Size in RAM: 88 bytes
Items: 3 | Size in RAM: 88 bytes
Items: 4 | Size in RAM: 88 bytes
Items: 5 | Size in RAM: 120 bytes
Items: 6 | Size in RAM: 120 bytes
Items: 7 | Size in RAM: 120 bytes
Items: 8 | Size in RAM: 120 bytes
Items: 9 | Size in RAM: 184 bytes
Items: 10 | Size in RAM: 184 bytes

Static Tuple Size for 10 items: 120 bytes


## Lesson 3: Why 16 Cores aren't faster (The GIL)

### The Problem
You have a CPU-heavy calculation for each truck. You tried using `threading` to use all 16 cores, but the script is just as slow as the single-threaded version.

### The "Aha!" Moment
The **Global Interpreter Lock (GIL)** ensures only one thread executes Python bytecode at a time. This makes threads great for I/O (waiting for a network) but useless for heavy math.

In [16]:
import threading
import time

def heavy_math(n):
    while n > 0:
        n -= 1

COUNT = 10**7

# Sequential execution
start = time.time()
heavy_math(COUNT*2)
print(f"Sequential Time: {time.time() - start:.2f}s")

# Threaded execution (Limited by GIL)
threads = []
for t in range(2):
    threads.append(threading.Thread(target=heavy_math, args=(COUNT,)))


start = time.time()
[t.start() for t in threads]
[t.join() for t in threads]

print(f"Threaded Time (GIL bottleneck): {time.time() - start:.2f}s")

Sequential Time: 0.37s
Threaded Time (GIL bottleneck): 0.33s


## Lesson 4: The Silent Leak (Circular References)

### The Problem
Your worker script runs for days, but the memory usage slowly creeps up until the OS kills it. You aren't storing data, so where is it going?

### The "Aha!" Moment
If Object A points to B, and B points to A, their reference counts never hit zero. Python's Garbage Collector (GC) has to find and destroy these "islands."

In [38]:
import gc

class Node:
    def __init__(self, name):
        self.name = name
        self.partner = None
    def __del__(self):
        # In a real leak, this never prints until GC runs
        pass

def create_leak():
    n1 = Node("A")
    n2 = Node("B")
    n1.partner = n2
    n2.partner = n1
    print("Cycle created and references deleted.")

gc.disable() # Let's simulate a busy system where GC hasn't triggered yet
create_leak()

print(f"Unreachable objects before GC: {len(gc.garbage)}")
found = gc.collect()
print(f"GC manually triggered. Found and cleared {found} objects.")
gc.enable()

Cycle created and references deleted.
Unreachable objects before GC: 0
GC manually triggered. Found and cleared 11 objects.
